In [2]:
# get_image_feature_vectors.py
#################################################
# Imports and function definitions
#################################################
# For running inference on the TF-Hub module with Tensorflow
import tensorflow as tf
import tensorflow_hub as hub
# For saving 'feature vectors' into a txt file
import numpy as np
# Glob for reading file names in a folder
import glob
import os.path

In [3]:

#################################################
#################################################
# This function:
# Loads the JPEG image at the given path
# Decodes the JPEG image to a uint8 W X H X 3 tensor
# Resizes the image to 224 x 224 x 3 tensor
# Returns the pre processed image as 224 x 224 x 3 tensor
#################################################
def load_img(path):
# Reads the image file and returns data type of string
 img = tf.io.read_file(path)
# Decodes the image to W x H x 3 shape tensor with type of uint8
 img = tf.io.decode_jpeg(img, channels=3)
# Resizes the image to 224 x 224 x 3 shape tensor
 img = tf.image.resize_with_pad(img, 224, 224)
# Converts the data type of uint8 to float32 by adding a new axis
 # img becomes 1 x 224 x 224 x 3 tensor with data type of float32
 # This is required for the mobilenet model we are using
 img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis, ...]
 
 return img
#################################################
# This function:
# Loads the mobilenet model in TF.HUB
# Makes an inference for all images stored in a local folder
# Saves each of the feature vectors in a file
#################################################
def get_image_feature_vectors():
 
 # Definition of module with using tfhub.dev
 module_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4"
 # Loads the module
 module = hub.load(module_handle)
# Loops through all images in a local folder
 for filename in glob.glob('D:/Documents/Semester3/ADM/Assignment 3/Pics/*.jpg'):
 
  print(filename)
# Loads and pre-process the image
  img = load_img(filename)
# Calculate the image feature vector of the img
  features = module(img)
# Remove single-dimensional entries from the 'features' array  
  feature_set = np.squeeze(features)
 
  # Saves the image feature vectors into a file for later use
  outfile_name = os.path.basename(filename) + ".npz"
 
  #out_path = os.path.join('D:/Documents/Semester3/ADM/Assignment 3/Images_Scraped/',outfile_name)
  out_path = os.path.join('D:/Documents/Semester3/ADM/Assignment 3/Pictures_Scraped/',outfile_name)
# Saves the 'feature_set' to a text file
  np.savetxt(out_path, feature_set, delimiter=',')
get_image_feature_vectors()

D:/Documents/Semester3/ADM/Assignment 3/Pics\10531_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\11485_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\14058_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\15759_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\18337_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\23158_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\26567_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\26567_1.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\26567_2.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\2804_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\545_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\6095_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\6095_1.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\6095_2.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\6172_0.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\6172_1.jpg
D:/Documents/Semester3/ADM/Assignment 3/Pics\6172_2.jpg
D:/Documents/Semester3/ADM/Assignment 3/

In [4]:
#################################################
# Imports and function definitions
#################################################
# Numpy for loading image feature vectors from file
import numpy as np
# Time for measuring the process time
import time
# Glob for reading file names in a folder
import glob
import os.path
# json for storing data in json file
import json

In [5]:

#################################################
# This script reads image feature vectors from a folder
# and saves the image similarity scores in json file
# by Erdem Isbilen - December/2019
#################################################

#################################################
# Imports and function definitions
#################################################

# Numpy for loading image feature vectors from file
import numpy as np

# Time for measuring the process time
import time

# Glob for reading file names in a folder
import glob
import os.path

# json for storing data in json file
import json

# Annoy and Scipy for similarity calculation
from annoy import AnnoyIndex
from scipy import spatial
#################################################

#################################################
# This function reads from 'image_data.json' file
# Looks for a specific 'filename' value
# Returns the product id when product image names are matched 
# So it is used to find product id based on the product image name
#################################################
def match_id(filename):
    print(filename)
    product_id = '_'.join(filename.split('_')[:-1])
    return product_id
#################################################

#################################################
# This function; 
# Reads all image feature vectores stored in /feature-vectors/*.npz
# Adds them all in Annoy Index
# Builds ANNOY index
# Calculates the nearest neighbors and image similarity metrics
# Stores image similarity scores with productID in a json file
#################################################
def cluster():

    start_time = time.time()
  
    print("---------------------------------")
    print ("Step.1 - ANNOY index generation - Started at %s" %time.ctime())
    print("---------------------------------")

    # Defining data structures as empty dict
    file_index_to_file_name = {}
    file_index_to_file_vector = {}
    file_index_to_product_id = {}

    # Configuring annoy parameters
    dims = 1792
    n_nearest_neighbors = 20
    trees = 10000

    # Reads all file names which stores feature vectors 
    allfiles = glob.glob('D:/Documents/Semester3/ADM/Assignment 3/Images/Images_Scraped/*.npz')

    t = AnnoyIndex(dims, metric='angular')

    for file_index, i in enumerate(allfiles):
    
    # Reads feature vectors and assigns them into the file_vector 
        file_vector = np.loadtxt(i)

        # Assigns file_name, feature_vectors and corresponding product_id
        file_name = os.path.basename(i).split('.')[0]
        file_index_to_file_name[file_index] = file_name
        file_index_to_file_vector[file_index] = file_vector
        file_index_to_product_id[file_index] = match_id(file_name)

        # Adds image feature vectors into annoy index   
        t.add_item(file_index, file_vector)

        print("---------------------------------")
        print("Annoy index     : %s" %file_index)
        print("Image file name : %s" %file_name)
        print("Product id      : %s" %file_index_to_product_id[file_index])
        print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))


    # Builds annoy index
    t.build(trees)

    print ("Step.1 - ANNOY index generation - Finished")
    print ("Step.2 - Similarity score calculation - Started ") 
  
    named_nearest_neighbors = []

  # Loops through all indexed items
    for i in file_index_to_file_name.keys():

    # Assigns master file_name, image feature vectors and product id values
        master_file_name = file_index_to_file_name[i]
        master_vector = file_index_to_file_vector[i]
        master_product_id = file_index_to_product_id[i]

        # Calculates the nearest neighbors of the master item
        nearest_neighbors = t.get_nns_by_item(i, n_nearest_neighbors)

        # Loops through the nearest neighbors of the master item
        for j in nearest_neighbors:

            print(j)

            # Assigns file_name, image feature vectors and product id values of the similar item
            neighbor_file_name = file_index_to_file_name[j]
            neighbor_file_vector = file_index_to_file_vector[j]
            neighbor_product_id = file_index_to_product_id[j]

            # Calculates the similarity score of the similar item
            similarity = 1 - spatial.distance.cosine(master_vector, neighbor_file_vector)
            rounded_similarity = int((similarity * 10000)) / 10000.0

            # Appends master product id with the similarity score 
            # and the product id of the similar items
            named_nearest_neighbors.append({
            'similarity': rounded_similarity,
            'master_pi': master_product_id,
            'similar_pi': neighbor_product_id})

    print("---------------------------------") 
    print("Similarity index       : %s" %i)
    print("Master Image file name : %s" %file_index_to_file_name[i]) 
    print("Nearest Neighbors.     : %s" %nearest_neighbors) 
    print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))

  
    print ("Step.2 - Similarity score calculation - Finished ") 

  # Writes the 'named_nearest_neighbors' to a json file
    with open('nearest_neighbors.json', 'w') as out:
        json.dump(named_nearest_neighbors, out)

        print ("Step.3 - Data stored in 'nearest_neighbors.json' file ") 
        print("--- Prosess completed in %.2f minutes ---------" % ((time.time() - start_time)/60))

cluster()

---------------------------------
Step.1 - ANNOY index generation - Started at Wed Nov  4 11:57:39 2020
---------------------------------
10000377_0
---------------------------------
Annoy index     : 0
Image file name : 10000377_0
Product id      : 10000377
--- 0.00 minutes passed ---------
10000419_0
---------------------------------
Annoy index     : 1
Image file name : 10000419_0
Product id      : 10000419
--- 0.01 minutes passed ---------
10000419_1
---------------------------------
Annoy index     : 2
Image file name : 10000419_1
Product id      : 10000419
--- 0.01 minutes passed ---------
10002588_0
---------------------------------
Annoy index     : 3
Image file name : 10002588_0
Product id      : 10002588
--- 0.01 minutes passed ---------
10003514_0
---------------------------------
Annoy index     : 4
Image file name : 10003514_0
Product id      : 10003514
--- 0.01 minutes passed ---------
10003897_0
---------------------------------
Annoy index     : 5
Image file name : 1000

10029918_2
---------------------------------
Annoy index     : 56
Image file name : 10029918_2
Product id      : 10029918
--- 0.04 minutes passed ---------
10030108_0
---------------------------------
Annoy index     : 57
Image file name : 10030108_0
Product id      : 10030108
--- 0.04 minutes passed ---------
10030983_0
---------------------------------
Annoy index     : 58
Image file name : 10030983_0
Product id      : 10030983
--- 0.05 minutes passed ---------
10030983_1
---------------------------------
Annoy index     : 59
Image file name : 10030983_1
Product id      : 10030983
--- 0.05 minutes passed ---------
10033074_0
---------------------------------
Annoy index     : 60
Image file name : 10033074_0
Product id      : 10033074
--- 0.05 minutes passed ---------
10033985_0
---------------------------------
Annoy index     : 61
Image file name : 10033985_0
Product id      : 10033985
--- 0.05 minutes passed ---------
10035423_0
---------------------------------
Annoy index     : 6

10062769_0
---------------------------------
Annoy index     : 110
Image file name : 10062769_0
Product id      : 10062769
--- 0.08 minutes passed ---------
10065154_0
---------------------------------
Annoy index     : 111
Image file name : 10065154_0
Product id      : 10065154
--- 0.08 minutes passed ---------
1006588_0
---------------------------------
Annoy index     : 112
Image file name : 1006588_0
Product id      : 1006588
--- 0.08 minutes passed ---------
1006588_1
---------------------------------
Annoy index     : 113
Image file name : 1006588_1
Product id      : 1006588
--- 0.08 minutes passed ---------
1006588_2
---------------------------------
Annoy index     : 114
Image file name : 1006588_2
Product id      : 1006588
--- 0.08 minutes passed ---------
1006588_3
---------------------------------
Annoy index     : 115
Image file name : 1006588_3
Product id      : 1006588
--- 0.08 minutes passed ---------
100679_0
---------------------------------
Annoy index     : 116
Image

10096845_2
---------------------------------
Annoy index     : 166
Image file name : 10096845_2
Product id      : 10096845
--- 0.12 minutes passed ---------
10096845_3
---------------------------------
Annoy index     : 167
Image file name : 10096845_3
Product id      : 10096845
--- 0.12 minutes passed ---------
10097188_0
---------------------------------
Annoy index     : 168
Image file name : 10097188_0
Product id      : 10097188
--- 0.12 minutes passed ---------
10099845_0
---------------------------------
Annoy index     : 169
Image file name : 10099845_0
Product id      : 10099845
--- 0.12 minutes passed ---------
10103254_0
---------------------------------
Annoy index     : 170
Image file name : 10103254_0
Product id      : 10103254
--- 0.12 minutes passed ---------
10106328_0
---------------------------------
Annoy index     : 171
Image file name : 10106328_0
Product id      : 10106328
--- 0.12 minutes passed ---------
10106802_0
---------------------------------
Annoy index  

10164958_3
---------------------------------
Annoy index     : 220
Image file name : 10164958_3
Product id      : 10164958
--- 0.15 minutes passed ---------
10165073_0
---------------------------------
Annoy index     : 221
Image file name : 10165073_0
Product id      : 10165073
--- 0.15 minutes passed ---------
10166558_0
---------------------------------
Annoy index     : 222
Image file name : 10166558_0
Product id      : 10166558
--- 0.15 minutes passed ---------
10166558_1
---------------------------------
Annoy index     : 223
Image file name : 10166558_1
Product id      : 10166558
--- 0.15 minutes passed ---------
10166558_2
---------------------------------
Annoy index     : 224
Image file name : 10166558_2
Product id      : 10166558
--- 0.15 minutes passed ---------
10166558_3
---------------------------------
Annoy index     : 225
Image file name : 10166558_3
Product id      : 10166558
--- 0.15 minutes passed ---------
10168505_0
---------------------------------
Annoy index  

...

18579459_0
---------------------------------
Annoy index     : 8839
Image file name : 18579459_0
Product id      : 18579459
--- 5.35 minutes passed ---------
18583135_0
---------------------------------
Annoy index     : 8840
Image file name : 18583135_0
Product id      : 18583135
--- 5.35 minutes passed ---------
18583135_1
---------------------------------
Annoy index     : 8841
Image file name : 18583135_1
Product id      : 18583135
--- 5.35 minutes passed ---------
18583135_2
---------------------------------
Annoy index     : 8842
Image file name : 18583135_2
Product id      : 18583135
--- 5.35 minutes passed ---------
18583135_3
---------------------------------
Annoy index     : 8843
Image file name : 18583135_3
Product id      : 18583135
--- 5.35 minutes passed ---------
18584466_0
---------------------------------
Annoy index     : 8844
Image file name : 18584466_0
Product id      : 18584466
--- 5.35 minutes passed ---------
18585603_0
---------------------------------
Annoy i

18626331_0
---------------------------------
Annoy index     : 8892
Image file name : 18626331_0
Product id      : 18626331
--- 5.37 minutes passed ---------
18627933_0
---------------------------------
Annoy index     : 8893
Image file name : 18627933_0
Product id      : 18627933
--- 5.37 minutes passed ---------
18628081_0
---------------------------------
Annoy index     : 8894
Image file name : 18628081_0
Product id      : 18628081
--- 5.38 minutes passed ---------
18628081_1
---------------------------------
Annoy index     : 8895
Image file name : 18628081_1
Product id      : 18628081
--- 5.38 minutes passed ---------
18628081_2
---------------------------------
Annoy index     : 8896
Image file name : 18628081_2
Product id      : 18628081
--- 5.38 minutes passed ---------
18628081_3
---------------------------------
Annoy index     : 8897
Image file name : 18628081_3
Product id      : 18628081
--- 5.38 minutes passed ---------
18636061_0
---------------------------------
Annoy i

18688227_0
---------------------------------
Annoy index     : 8945
Image file name : 18688227_0
Product id      : 18688227
--- 5.41 minutes passed ---------
18689649_0
---------------------------------
Annoy index     : 8946
Image file name : 18689649_0
Product id      : 18689649
--- 5.41 minutes passed ---------
18691154_0
---------------------------------
Annoy index     : 8947
Image file name : 18691154_0
Product id      : 18691154
--- 5.41 minutes passed ---------
1869518_0
---------------------------------
Annoy index     : 8948
Image file name : 1869518_0
Product id      : 1869518
--- 5.41 minutes passed ---------
18696881_0
---------------------------------
Annoy index     : 8949
Image file name : 18696881_0
Product id      : 18696881
--- 5.41 minutes passed ---------
18699087_0
---------------------------------
Annoy index     : 8950
Image file name : 18699087_0
Product id      : 18699087
--- 5.41 minutes passed ---------
1869981_0
---------------------------------
Annoy index

1873983_2
---------------------------------
Annoy index     : 8999
Image file name : 1873983_2
Product id      : 1873983
--- 5.44 minutes passed ---------
1873983_3
---------------------------------
Annoy index     : 9000
Image file name : 1873983_3
Product id      : 1873983
--- 5.44 minutes passed ---------
18740050_0
---------------------------------
Annoy index     : 9001
Image file name : 18740050_0
Product id      : 18740050
--- 5.44 minutes passed ---------
18740218_0
---------------------------------
Annoy index     : 9002
Image file name : 18740218_0
Product id      : 18740218
--- 5.45 minutes passed ---------
18740218_1
---------------------------------
Annoy index     : 9003
Image file name : 18740218_1
Product id      : 18740218
--- 5.45 minutes passed ---------
18740555_0
---------------------------------
Annoy index     : 9004
Image file name : 18740555_0
Product id      : 18740555
--- 5.45 minutes passed ---------
18740555_1
---------------------------------
Annoy index  

KeyboardInterrupt: 